In [1]:
!pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 1.2 MB/s eta 0:00:00a 0:00:01


In [2]:
from kafka import KafkaConsumer, KafkaProducer
import json
import uuid

def connect_kafka_producer(servers):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=servers, api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [3]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))

In [4]:
import random
import time

def produce_xy(producer, topic_name):
    while True:
        name = f'Mark {random.randint(0,22)}'
        message = json.dumps({"name": name})
        print(name)
        publish_message(producer, topic_name, str(uuid.uuid4()), message)
        time.sleep(1)

In [5]:
def consume_xy(consumer, topic_name):

    for msg in consumer:
        print(msg.key.decode("utf-8"), msg.value)

In [6]:
server1 = 'broker1:9093'
topic1 = "names"

producer = connect_kafka_producer(server1)

consumer = KafkaConsumer(topic1, 
                         auto_offset_reset='earliest',
                         bootstrap_servers=[server1], 
                         api_version=(0, 10), 
                         value_deserializer = json.loads,
                         consumer_timeout_ms=1000)

In [14]:
produce_xy(producer, topic1)

Mark 15
Message published successfully.
Mark 2
Message published successfully.


KeyboardInterrupt: 

In [15]:
consume_xy(consumer, topic1)

e7b73a19-b0c0-44af-8255-995661757591 {'name': 'Mark 15'}
1505ba3f-d1fa-4d41-8610-46d99404fa54 {'name': 'Mark 2'}
